In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1679589332000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detection_Data.csv', name='Occupancy_Detection_Data.csv', size=50968, modificationTime=1677683107000),
 FileInfo(path='dbfs:/FileStore/tables/TS021_2021_2.csv', name='TS021_2021_2.csv', size=497239, modificationTime=1679508416000),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1675271048000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv', name='clinicaltrial_2019.csv', size=42400056, modificationTime=1681666373000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.zip', name='clinicaltrial_2019.zip', size=9707871, modificationTime=1681659047000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020.csv', 

In [0]:
fileroot = "clinicaltrial_2021"

In [0]:
dbutils.fs.cp("/FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

Out[3]: True

In [0]:
import os
os.environ['fileroot'] = fileroot

In [0]:
%sh
unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2021.zip
  inflating: /tmp/clinicaltrial_2021.csv  


In [0]:
dbutils.fs.mkdirs("/FileStore/tables/" + fileroot)

Out[6]: True

In [0]:
dbutils.fs.mv("file:/tmp/" + fileroot + ".csv", "/FileStore/tables/", True)

Out[7]: True

In [0]:
#converting the csv file to dataframes
clinic_df1 = spark.read.option("header", True).option("escape",'\"').option("sep", "|").csv("/FileStore/tables/clinicaltrial_2021.csv")
pharma_df1 = spark.read.option("header", True).option("escape",'\"').csv("/FileStore/tables/pharma.csv")
#clinic_df1.show(50)
#pharma_df1.show(1)

In [0]:
#converting the dataframe to rdd
clinic_rdd=clinic_df1.rdd
pharma_rdd=pharma_df1.rdd

clinic_rdd.take(1)
pharma_rdd.take(1)

Out[9]: [Row(Company='Abbott Laboratories', Parent_Company='Abbott Laboratories', Penalty_Amount='$5,475,000', Subtraction_From_Penalty='$0', Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting='$5,475,000', Penalty_Year='2013', Penalty_Date='20131227', Offense_Group='government-contracting-related offenses', Primary_Offense='False Claims Act and related', Secondary_Offense='kickbacks and bribery', Description="Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the company's carotid, biliary and peripheral vascular products.", Level_of_Government='federal', Action_Type='agency action', Agency='Justice Department Civil Division', Civil/Criminal='civil', Prosecution_Agreement=None, Court=None, Case_ID=None, Private_Litigation_Case_Title=None, Lawsuit_Resolution=None, Facility_State=None, City=None, Address=None, Zip=None, NAICS_Code=None, NAICS_Translation=None, HQ_Country_of_Par

In [0]:
# Answer to Question 1.
clinic_rdd.filter(lambda row: row.Type is not None) \
    .map(lambda row: (row.Type, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda row : row[1]) \
    .reduce(lambda x, y: x + y)

Out[10]: 387261

In [0]:
# Answer to Question2

clinic_rdd.filter(lambda row: row.Type is not None) \
    .map(lambda row: (row.Type, 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .map(lambda row: (row[1], row[0])) \
    .sortByKey(False).take(5)

Out[11]: [(301472, 'Interventional'),
 (77540, 'Observational'),
 (8180, 'Observational [Patient Registry]'),
 (69, 'Expanded Access')]

In [0]:
# Answer to Question 3

clinic_rdd.filter(lambda row: row.Conditions is not None) \
   .filter(lambda row: row.Id is not None) \
   .map(lambda row: (row.Id, row.Conditions)) \
   .flatMapValues(lambda x: x.split(",")) \
   .map(lambda row: (row[1].strip(),1)) \
   .reduceByKey(lambda x, y: x + y) \
   .map(lambda row: (row[1], row[0])) \
   .sortByKey(False) \
   .take(5)


Out[12]: [(13389, 'Carcinoma'),
 (11080, 'Diabetes Mellitus'),
 (9371, 'Neoplasms'),
 (8640, 'Breast Neoplasms'),
 (8032, 'Syndrome')]

In [0]:
# Anwer to Question 4

cli_d=clinic_rdd.filter(lambda row: row.Sponsor is not None) \
  .filter(lambda row: row.Type is not None) \
  .map(lambda row: (row.Sponsor.strip(),row.Type.strip()))

pha_d=pharma_rdd.filter(lambda row: row.Parent_Company is not None) \
  .filter(lambda row: row.Company is not None) \
  .map(lambda row: (row.Parent_Company.strip(),row.Company.strip()))

cli_d.subtractByKey(pha_d) \
   .map(lambda row: (row[0],1)) \
   .reduceByKey(lambda x, y: x + y) \
   .map(lambda row: (row[1], row[0])) \
   .sortByKey(False) \
   .take(10)

Out[13]: [(3218, 'National Cancer Institute (NCI)'),
 (2414, 'M.D. Anderson Cancer Center'),
 (2369, 'Assistance Publique - Hôpitaux de Paris'),
 (2300, 'Mayo Clinic'),
 (2243, 'Merck Sharp & Dohme Corp.'),
 (2154, 'Assiut University'),
 (2088, 'Novartis Pharmaceuticals'),
 (1971, 'Massachusetts General Hospital'),
 (1928, 'Cairo University'),
 (1828, 'Hoffmann-La Roche')]

In [0]:
#question(5) 1
clinic_rdd.filter(lambda row: row.Status is not None) \
   .filter(lambda row: row.Completion is not None) \
   .filter(lambda row: row.Status == 'Completed') \
   .map(lambda row: (row.Status,row.Completion,row.Completion[4:8])) \
   .filter(lambda row: row[2] == '2021') \
   .map(lambda row: (row[1].strip(),row[0])) \
   .groupByKey() \
   .map(lambda row: (row[0], len(row[1]))) \
   .sortByKey() \
   .take(15)

Out[14]: [('Apr 2021', 967),
 ('Aug 2021', 700),
 ('Feb 2021', 934),
 ('Jan 2021', 1131),
 ('Jul 2021', 819),
 ('Jun 2021', 1094),
 ('Mar 2021', 1227),
 ('May 2021', 984),
 ('Oct 2021', 187),
 ('Sep 2021', 528)]

In [0]:
# Answer to Question 5 (Arranged in a monthly order)
import calendar as c
month_map = {v: k for k, v in enumerate(c.month_abbr)}
myrdd = clinic_rdd.filter(lambda row: row.Status is not None and row.Completion is not None)
myrdd = myrdd.filter(lambda row: row.Status == 'Completed' and row.Completion.endswith('2021'))
myrdd = myrdd.map(lambda row: (row.Completion[0:3], 1)) \
             .reduceByKey(lambda x, y: x + y)
myrdd = myrdd.sortBy(lambda x: month_map[x[0]])
myrdd.collect()

Out[15]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

In [0]:
 # Answer to Question 5 (Arrange without including the Month Order)
  
myList = myrdd.collect()
my_list_dict = dict(myList)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
newList = []
for m in months:
  if m in my_list_dict:
    newList.append(my_list_dict[m])
print(newList)

[1131, 934, 1227, 967, 1094, 819, 700, 528, 187]


In [0]:
# Extra Features 1

import pandas as pd
result_rdd = clinic_rdd.map(lambda row: (row.Sponsor, row.Start.split('/')[-1], 1)) \
                 .groupBy(lambda row: row[0]) \
                 .map(lambda row: (row[0], sum([trial[2] for trial in row[1]]), len(set([trial[1] for trial in row[1]])))) \
                 .map(lambda row: (row[0], row[1] / row[2])) \
                 .sortBy(lambda row: row[1], ascending=False)

result_df = pd.DataFrame(result_rdd.collect(), columns=['Sponsor', 'TrialsPerYear'])
print(result_df.head(10))


                                             Sponsor  TrialsPerYear
0                                  Assiut University      18.730435
1                                   Cairo University      13.027027
2                                    GlaxoSmithKline      11.608247
3            Assistance Publique - Hôpitaux de Paris       9.748971
4                                        AstraZeneca       9.714801
5                                             Pfizer       9.412811
6  International Institute of Rescue Research and...       9.250000
7                    National Cancer Institute (NCI)       8.604278
8                                        Mayo Clinic       8.424908
9                           Novartis Pharmaceuticals       8.156250


In [0]:
# This code maps each row of the RDD to a tuple of (condition, 1), then reduces by key to count the occurrences of each condition, sorts the result in descending order by the count, and finally takes the first element, which should be the condition with the highest count.

clinic_rdd.map(lambda row: (row.Conditions, 1)) \
  .reduceByKey(lambda a, b: a + b) \
  .sortBy(lambda row: row[1], ascending=False) \
  .take(1)

Out[18]: [(None, 65131)]

In [0]:
# Extra Features
clinic_rdd.map(lambda row: (row.Sponsor, row.Start.split('/')[-1], 1)) \
                 .groupBy(lambda row: row[0]) \
                 .map(lambda row: (row[0], sum([trial[2] for trial in row[1]]), len(set([trial[1] for trial in row[1]])))) \
                 .map(lambda row: (row[0], row[1] / row[2])) \
                 .sortBy(lambda row: row[1], ascending=False) \
                 .take(10)

Out[19]: [('Assiut University', 18.730434782608697),
 ('Cairo University', 13.027027027027026),
 ('GlaxoSmithKline', 11.608247422680412),
 ('Assistance Publique - Hôpitaux de Paris', 9.748971193415638),
 ('AstraZeneca', 9.71480144404332),
 ('Pfizer', 9.412811387900355),
 ('International Institute of Rescue Research and Education', 9.25),
 ('National Cancer Institute (NCI)', 8.60427807486631),
 ('Mayo Clinic', 8.424908424908425),
 ('Novartis Pharmaceuticals', 8.15625)]